In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import time as time

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPU devices available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.optimizer.set_jit(enabled=True)
# tf.config.optimizer.set_experimental_options({"loop_optimization":True,"layout_optimizer":True})

Num of GPU devices available:  1


In [3]:
import requests
URL = "https://visualgenome.org/api/v0/images/"


def download_visual_genome_image(image_id, folder):
    """ Downloads image from the visual genome dataset

    :param image_id: id of the image to download
    :type image_id: str
    :param folder: where to download the image
    :return:
    """
    r = requests.get(url=URL + image_id)
    data = r.json()

    print(f"Downloading image: {data['url']}")

    img_data = requests.get(data['url']).content
    with open(folder + f'/{image_id}.jpg', 'wb') as handler:
        handler.write(img_data)

In [4]:
import os

def create_descriptor_features(image_files):
    """Create features for images with SIFT descriptor

    :param image_files: list of images to be processed
    :type image_files: list(str)
    :return: numpy array of the created features
    :rtype: np.array
    """
    trainer = cv2.BOWKMeansTrainer(clusterCount=100)
    sift = cv2.xfeatures2d.SIFT_create()
    matcher = cv2.FlannBasedMatcher_create()
    bow_extractor = cv2.BOWImgDescriptorExtractor(sift, matcher)

    print('Creating dictionary')
    if os.path.exists('data/dictionary.npy'):
        dictionary = np.load('data/dictionary.npy')
    else:
        for filename in image_files:
            file = f'data/visual_genome/{filename.lower()}'
            img = cv2.imread(file)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            key_points, desc_obj = sift.detectAndCompute(img, mask=None)
            trainer.add(desc_obj)

        dictionary = trainer.cluster()
        np.save('data/dictionary.npy', dictionary)

    bow_extractor.setVocabulary(dictionary)

    feature_data = np.zeros(shape=(len(image_files), dictionary.shape[0]),
                            dtype=np.float32)

    print('Extract features')
    for i, filename in zip(range(len(image_files)), image_files):
        file = f'data/visual_genome/{filename.lower()}'
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        points = sift.detect(img)
        feature_data[i] = bow_extractor.compute(img, points)
    return feature_data

In [5]:
import pickle
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# tensorflow.contrib.keras.api.keras.applications.vgg16 from original code

def load_vgg16(fc):
    """ Creates VGG16 model.

    :param fc: fully connected layer as output layer if true
    :type fc: bool
    :return: instance of VGG16 keras model
    :rtype: keras.Model
    """
    base_model = VGG16(include_top=True, weights='imagenet', input_shape=(224, 224, 3))
    if fc:
        model = Model(inputs=base_model.input, outputs=base_model.get_layer(name='fc2').output)
    else:
        model = Model(inputs=base_model.input, outputs=base_model.get_layer(name='block5_pool').output)
    model.trainable = False
    return model


def create_features(image_id, model):
    """ Creates features with VGG16 model for given image.

    :param image_id: id of the image
    :type image_id: str
    :param model: VGG16 model
    :type model: keras.Model
    :return: features of the image
    :rtype: numpy.array
    """
    img = cv2.resize(cv2.imread(f'./data/visual_genome/{image_id}.jpg'), (224, 224))
    features = model.predict(preprocess_input(np.expand_dims(img.astype(np.float32), axis=0)))
    return features[0]


def create_features_parallel(image_ids, model):
    """ Creates features with VGG16 model for given image.

    :param image_ids: ids of the images
    :type image_ids: list
    :param model: VGG16 model
    :type model: keras.Model
    :return: features of the image
    :rtype: numpy.array
    """
    input_f = []
    for image_id in image_ids:
        print("START", image_id)  # last printed DONE 2414570
        img = cv2.imread("./data/visual_genome/" + str(image_id) + '.jpg')
        img = cv2.resize(img, (224, 224))
        input_f.append(img.astype(np.float32))

    features = model.predict(preprocess_input(np.array(input_f)))
    return features


def load_vgg16_features(image_id, fc):
    """ Loads VGG16 features for the image with given id. It assumes that the features are already created.

    :param image_id: id of the image
    :type image_id: str
    :param fc: use features from fully connected layer if true
    :type fc: bool
    :return: features of the image
    :rtype: numpy.array
    """
    if fc:
        with open(f'./Lab05_Output/dataset/features/vgg16/{image_id}.pkl', 'rb') as f:
            features = pickle.load(f)
    else:
        with open(f'./Lab05_Output/dataset/features/vgg16-conv/{image_id}.pkl', 'rb') as f:
            features = pickle.load(f)
    return features


def create_vgg16_features_parallel(image_ids, fc):
    """ Creates VGG16 features for images with given ids. Features are saved to a file

    :param image_ids: image ids
    :type image_ids: numpy.array
    :param fc: use features from fully connected layer if true
    :type fc: bool
    """
    vgg_16_model = load_vgg16(fc)
    images = []
    for i, image_id in zip(tqdm(list(range(len(image_ids)))), image_ids):
        if fc:
            features_path = f'./Lab05_Output/dataset/features/vgg16/{image_id}.pkl'
        else:
            features_path = f'./Lab05_Output/dataset/features/vgg16-conv/{image_id}.pkl'
        if not os.path.exists(features_path):
            images.append(image_id)
        if len(images) == 16 or i == len(image_ids) - 1:
            features = create_features_parallel(images, vgg_16_model)
            for im_id, feats in zip(images, features):
                if fc:
                    with open(f'./Lab05_Output/dataset/features/vgg16/{im_id}.pkl', 'wb') as f:
                        pickle.dump(feats, f)
                else:
                    with open(f'./Lab05_Output/dataset/features/vgg16-conv/{im_id}.pkl', 'wb') as f:
                        pickle.dump(feats, f)
            images = []


def create_vgg16_features(image_ids, fc):
    """ Creates VGG16 features for images with given ids. Features are saved to a file

    :param image_ids: image ids
    :type image_ids: numpy.array
    :param fc: use features from fully connected layer if true
    :type fc: bool
    """
    vgg_16_model = load_vgg16(fc)
    for _, image_id in zip(tqdm(list(range(len(image_ids)))), image_ids):
        if fc:
            if not os.path.exists(f'./Lab05_Output/dataset/features/vgg16/{image_id}.pkl'):
                features = create_features(image_id, vgg_16_model)
                with open(f'./Lab05_Output/dataset/features/vgg16/{image_id}.pkl', 'wb') as f:
                    pickle.dump(features, f)
        else:
            if not os.path.exists(f'./Lab05_Output/dataset/features/vgg16-conv/{image_id}.pkl'):
                features = create_features(image_id, vgg_16_model)
                with open(f'./Lab05_Output/dataset/features/vgg16-conv/{image_id}.pkl', 'wb') as f:
                    pickle.dump(features, f)

### Zad 1 - extracting data, train_test_split, simple RandomForestClassifier for openCV features extraction with BOWImgDescriptorExtractor
All images downloaded

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

vg_objects = pd.read_csv('./data/visual_genome_objects.csv')
encoder = LabelEncoder()
encoder.fit(vg_objects["Class"])
encoded_Y = encoder.transform(vg_objects["Class"])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)
X_train, x_test, Y_train, y_test = train_test_split(vg_objects, dummy_y, test_size=0.2,random_state=42)
X_test, x_val, Y_test, y_val = train_test_split(x_test,y_test, test_size=0.2, random_state=42)
classes = set(vg_objects["Class"])

In [7]:
train_np = X_train.to_numpy()
test_np = X_test.to_numpy()
val_np = x_val.to_numpy()
for arr in (train_np, test_np, val_np):
    arr[arr == "building"] = 0
    arr[arr == "car"] = 1
    arr[arr == "chair"] = 2
    arr[arr == "desk"] = 3
    arr[arr == "glass"] = 4
    arr[arr == "man"] = 5
    arr[arr == "road"] = 6
    arr[arr == "shelf"] = 7
    arr[arr == "tree"] = 8
    arr[arr == "window"] = 9

In [7]:
classes_dc = { 0:"building", 1:"car", 2:"chair", 3:"desk", 4:"glass", 5:"man", 6:"road", 7:"shelf", 8:"tree", 9:"window"}

In [8]:
import shutil
# X_train_lst = list(X_train)
# x_val_lst = list(x_val)
# x_test_lst = list(x_test)
def create_seperate_class_folder(class_name):
    os.mkdir(f'Lab05_Output/dataset/genReadyImages/test/{class_name}')
    os.mkdir(f'Lab05_Output/dataset/genReadyImages/train/{class_name}')
    os.mkdir(f'Lab05_Output/dataset/genReadyImages/val/{class_name}')
def create_dest(ttv, class_name, im_id):
        cl_name_str = classes_dc[class_name]
        if ttv == 0:
            return f'Lab05_Output/dataset/genReadyImages/train/{cl_name_str}/{str(im_id)}.jpg'
        if ttv == 1:
            return f'Lab05_Output/dataset/genReadyImages/test/{cl_name_str}/{str(im_id)}.jpg'
        if ttv == 2:
            return f'Lab05_Output/dataset/genReadyImages/val/{cl_name_str}/{str(im_id)}.jpg'


def populate_with_images(file_list):
    for file in file_list:
        src = f'data/visual_genome/{str(file[2])}.jpg'
        dest = create_dest(file[0], file[1], file[2])
        shutil.copyfile(src,dest)
from numba import jit, cuda
# @jit(nopython=True)
def create_np_list(train_np_func, test_np_func, val_np_func):
    file_list = np.zeros((5000,3),dtype=int)
    print(file_list.shape, "last index", file_list[4999,0])
    arr_iter = 0
    print("SUCCESS START CREATE")
    for i in range(train_np_func.shape[0]):
        file_id = train_np_func[i,0]
        cl_id = train_np_func[i,1]
        file_list[arr_iter,0] = 0
        file_list[arr_iter,1] = cl_id
        file_list[arr_iter,2] = file_id
        arr_iter = arr_iter + 1
    for i in range(test_np_func.shape[0]):
        file_id = test_np_func[i,0]
        cl_id = test_np_func[i,1]
        file_list[arr_iter,0] = 1
        file_list[arr_iter,1] = cl_id
        file_list[arr_iter,2] = file_id
        arr_iter = arr_iter + 1
    for i in range(val_np_func.shape[0]):
        file_id = val_np_func[i,0]
        cl_id = val_np_func[i,1]
        file_list[arr_iter,0] = 2
        file_list[arr_iter,1] = cl_id
        file_list[arr_iter,2] = file_id
        arr_iter = arr_iter + 1
    return file_list

    # for im_id, cl in zip(relevant_ids["Image ID"],):
    #     src = f'data/visual_genome/{str(im_id)}.jpg'
    #     if im_id in X_train_lst:
    #         dest = f'Lab05_Output/dataset/genReadyImages/train/{class_name}/{str(im_id)}.jpg'
    #         shutil.copyfile(src,dest)
    #     if im_id in x_val_lst:
    #         dest = f'Lab05_Output/dataset/genReadyImages/val/{class_name}/{str(im_id)}.jpg'
    #         shutil.copyfile(src,dest)
    #     if im_id in x_test_lst:
    #         dest = f'Lab05_Output/dataset/genReadyImages/test/{class_name}/{str(im_id)}.jpg'
    #         shutil.copyfile(src,dest)
    #     if im_id not in X_train_lst and im_id not in x_val_lst and im_id not in x_test_lst:
    #         print("Strange for ", im_id)
def getNames(file):
    filename = os.fsdecode(file)
    img_name = filename.split(".")[0]
    return img_name
class_names = set(vg_objects["Class"])
if os.path.isdir("Lab05_Output/dataset/genReadyImages/train/car") is False:
    # for cl_nm in class_names:
    #     create_seperate_class_folder(cl_nm)
    start = time.time()
    # with tf.device('/device:GPU:0'):
    print("Start")
    file_lst = create_np_list(train_np[:].astype(np.int),test_np[:].astype(np.int),val_np[:].astype(np.int))
    print("cuda function: ", time.time() - start)
    print("Copy files")
    populate_with_images(file_lst)
directory = os.fsencode("./data/visual_genome")
lst_imgs = os.listdir(directory)
img_names = list(map(getNames,lst_imgs))
# for imgID in list(set(vg_objects['Image ID'])):
#     if str(imgID) not in img_names:
#         download_visual_genome_image(str(imgID), './data/visual_genome/')

In [10]:
import time as time
with tf.device('/device:GPU:0'):
    # print(list(map(lambda x: x + ".jpg",img_names)))
    start = time.time()
    image_files = list(map(lambda x: x + ".jpg",img_names))
    descriptor_features = create_descriptor_features(image_files)
    print("End time: ", time.time() - start)
    # descriptor_features

Creating dictionary
Extract features
End time:  518.2893946170807


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
images_path_train = './Lab05_Output/dataset/genReadyImages/train'
images_path_test = './Lab05_Output/dataset/genReadyImages/test'
images_path_val = './Lab05_Output/dataset/genReadyImages/val'
classes = list(class_names)
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=images_path_train,target_size=(244,244),classes=classes)
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=images_path_test,target_size=(244,244),classes=classes,shuffle=False,batch_size=20)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=images_path_val,target_size=(244,244),classes=classes)


assert train_batches.n == len(X_train)
assert test_batches.n ==  len(x_val)
assert val_batches.n == len(X_test)

Found 4000 images belonging to 10 classes.
Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [12]:
descriptor_features_withID = {}
for imgId, feat in zip(img_names, descriptor_features):
    descriptor_features_withID[imgId] = feat
# list(vg_objects["Image ID"])
# list(set(vg_objects["Image ID"]))

In [20]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()
RFC.fit([descriptor_features_withID[str(val)] for val in X_train['Image ID']],Y_train)
# y_pred = RFC.predict([descriptor_features_withID[str(val)] for val in x_test])
# print("Precision score: ", precision_score(y_test, y_pred, average='macro'))
score = RFC.score([descriptor_features_withID[str(val)] for val in x_test['Image ID']],y_test)
print("Score: ", score)
 # Is it really that bad or I am doing something really wrong ?

Score:  0.0


### Zad 2  - VGG 16

In [21]:
start = time.time()
modelVGG16 = load_vgg16(fc=True)
imgFeatures = {}

with tf.device('/device:GPU:0'):
    for imgID in img_names:
        feat = create_features(imgID, modelVGG16)
        imgFeatures[imgID] = feat
print("Needed: ", time.time() - start)

Needed:  176.33413195610046


In [ ]:
# pickle
# imgFeat_save = create_vgg16_features(img_names,True)
start = time.time()
print("pickles ready")
# with tf.device('/device:GPU:0'):
#     create_vgg16_features_parallel(img_names,True)
print("Needed: ", time.time() - start)

In [23]:
RFC = RandomForestClassifier()
RFC.fit([imgFeatures[str(val)] for val in X_train['Image ID']], Y_train)
score = RFC.score([imgFeatures[str(val)] for val in x_test['Image ID']],y_test)
print("Score: ", score)

Score:  0.048


In [24]:
imgFeat_save = {}
for imgID in img_names:
    imgFeat_save[imgID] = load_vgg16_features(imgID,True)
RFC = RandomForestClassifier()
RFC.fit([imgFeat_save[str(val)] for val in X_train['Image ID']], Y_train)
score = RFC.score([imgFeat_save[str(val)] for val in x_test['Image ID']],y_test)
print("Score: ", score)

Score:  0.05


### Zad 3

In [ ]:
from tensorflow.keras.layers import Input
if os.path.isdir("Lab05_Output/dataset/features/vgg16-conv") is False:
    with tf.device('/device:GPU:0'):
        os.chdir("Lab05_Output/dataset/features/")
        os.mkdir("vgg16-conv")
        os.chdir("../../../")
        create_vgg16_features_parallel(img_names,False)

##### Read all the pre trained features from VGG16 model

In [ ]:
imgFeat_save = {}
for imgID in img_names:
    imgFeat_save[imgID] = load_vgg16_features(imgID,False)

In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow import metrics
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score
num_classes = len(set(vg_objects["Class"]))

In [11]:
device = cuda.get_current_device()
device.reset()
if os.path.isfile('./Lab05_Output/models/pre_trained_with_vgg16.h5') is False:
    # base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg', classes=num_classes) # classifier_activation='Relu' MAKE SURE TO TRY IT
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg') # classifier_activation='Relu' MAKE SURE TO TRY IT
    x = base_model.output
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(4096,activation='relu')(x)
    output = Dense(10,activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    for layer in model.layers[:-13]:
        layer.trainable = False
    # model = Sequential(
    #     base_model.layers[:-3]
    #     +
    #     [
    #     Dense(4096, activation='relu'),
    #     Dropout(0.5),
    #     Dense(4096, activation='relu'),
    #     Dense(num_classes,activation='softmax')
    # ])
    model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# Not needed for now
# Input to base model is resized img (244, 244, 3)
# input_f = {}
# for image_id in img_names:  # last printed DONE 2414570
#     img = cv2.imread("./data/visual_genome/" + image_id + '.jpg')
#     img = cv2.resize(img, (224, 224))
#     input_f[image_id] = img.astype(np.float32)

In [12]:
if os.path.isfile('./Lab05_Output/models/pre_trained_with_vgg16.h5') is False:
    start = time.time()
    with tf.device('/device:GPU:0'):
        metrics = [metrics.Accuracy, metrics.Precision]
        model.compile(metrics=["accuracy"], loss="categorical_crossentropy",optimizer=Adam(learning_rate=1e-4))
        history = model.fit(x=train_batches,
            epochs=10, validation_data=val_batches, verbose=2)
    print("End: ", time.time() - start)
    model.save('./Lab05_Output/models/pre_trained_with_vgg16.h5')
else:
    model = load_model('./Lab05_Output/models/pre_trained_with_vgg16.h5')

Epoch 1/10
125/125 - 57s - loss: 2.2225 - accuracy: 0.1650 - val_loss: 2.1652 - val_accuracy: 0.1637
Epoch 2/10
125/125 - 41s - loss: 2.0250 - accuracy: 0.2062 - val_loss: 1.9269 - val_accuracy: 0.2050
Epoch 3/10
125/125 - 41s - loss: 1.9139 - accuracy: 0.2095 - val_loss: 1.8872 - val_accuracy: 0.2113
Epoch 4/10
125/125 - 41s - loss: 1.8591 - accuracy: 0.2292 - val_loss: 1.8671 - val_accuracy: 0.2113
Epoch 5/10
125/125 - 42s - loss: 1.8264 - accuracy: 0.2503 - val_loss: 1.8636 - val_accuracy: 0.2338
Epoch 6/10
125/125 - 43s - loss: 1.7809 - accuracy: 0.2630 - val_loss: 1.9623 - val_accuracy: 0.2000
Epoch 7/10
125/125 - 43s - loss: 1.7376 - accuracy: 0.2835 - val_loss: 1.8852 - val_accuracy: 0.2387
Epoch 8/10
125/125 - 43s - loss: 1.6985 - accuracy: 0.3045 - val_loss: 1.9782 - val_accuracy: 0.2250
Epoch 9/10
125/125 - 43s - loss: 1.6479 - accuracy: 0.3223 - val_loss: 1.9591 - val_accuracy: 0.2637
Epoch 10/10
125/125 - 43s - loss: 1.5998 - accuracy: 0.3420 - val_loss: 1.9422 - val_accura

In [13]:
# Make some predictions and evaluate
predictions = model.predict(x=test_batches, verbose=0)
cl = test_batches.classes
pred_int = np.argmax(predictions, axis=-1)
acurr_score = accuracy_score(pred_int,cl)
prec_score = precision_score(pred_int,cl, average="macro")
print("Custom DL model ----> acurracy: ", acurr_score, " ,  precision: ", np.round(prec_score, 3))

Custom DL model ----> acurracy:  0.115  ,  precision:  0.113


### Zad4 - building Deep Learning Model

In [11]:
# del model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)\
    .flow_from_directory(directory=images_path_train,target_size=(244,244),classes=classes,batch_size=20)
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)\
    .flow_from_directory(directory=images_path_test,target_size=(244,244),classes=classes,batch_size=20,shuffle=False)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)\
    .flow_from_directory(directory=images_path_val,target_size=(244,244),classes=classes,batch_size=20)


assert train_batches.n == len(X_train)
assert test_batches.n ==  len(x_val)
assert val_batches.n == len(X_test)

Found 4000 images belonging to 10 classes.
Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [12]:
device = cuda.get_current_device()
device.reset()
from tensorflow.keras.layers import Conv2D, MaxPool2D
if os.path.isfile('./Lab05_Output/models/custom_CNN.h5') is False:
    with tf.device('/device:GPU:0'):
        base_model = MobileNetV2(weights='imagenet')
        x = base_model.layers[-2].output
        fc1 = Dense(2048, activation='relu')(x)
        dp = Dropout(0.5)(fc1)
        fc2 = Dense(2048, activation='relu')(dp)
        output = Dense(num_classes,activation='softmax')(fc2)
        dl_model = Model(inputs=base_model.input, outputs=output)
        for layer in dl_model.layers[:-24]:
            layer.trainable = False
        # dl_model = Sequential([
        #     Conv2D(filters=32,kernel_size=(3,3), activation='relu', padding='same', input_shape=(244,244,3)),
        #     Conv2D(filters=32,kernel_size=(3,3), activation='relu', padding='same'),
        #     MaxPool2D(pool_size=(2,2),strides=2),
        #     Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding='same'),
        #     Conv2D(filters=64,kernel_size=(3,3), activation='relu', padding='same'),
        #     MaxPool2D(pool_size=(2,2),strides=2),
        #     Dense(512, activation='relu'),
        #     Dropout(0.5),
        #     Dense(512, activation='relu'),
        #     Flatten(),
        #     Dense(num_classes,activation='softmax')
        # ])
else:
    dl_model = load_model('./Lab05_Output/models/custom_CNN.h5')

dl_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
if os.path.isfile('./Lab05_Output/models/custom_CNN.h5') is False:
    with tf.device('/device:GPU:0'):
        dl_model.compile(optimizer=Adam(learning_rate=1e-4),loss='categorical_crossentropy', metrics=['accuracy'])
        dl_model.fit(x=train_batches,validation_data=val_batches,epochs=20,verbose=2)
        # save the model
        dl_model.save('./Lab05_Output/models/custom_CNN.h5')

Epoch 1/30
200/200 - 74s - loss: 1.8823 - accuracy: 0.2937 - val_loss: 2.3128 - val_accuracy: 0.3000
Epoch 2/30
200/200 - 18s - loss: 1.6008 - accuracy: 0.3983 - val_loss: 2.2617 - val_accuracy: 0.3225
Epoch 3/30
200/200 - 17s - loss: 1.4655 - accuracy: 0.4453 - val_loss: 2.0603 - val_accuracy: 0.3212
Epoch 4/30
200/200 - 18s - loss: 1.3316 - accuracy: 0.4955 - val_loss: 2.2540 - val_accuracy: 0.2975
Epoch 5/30
200/200 - 18s - loss: 1.2129 - accuracy: 0.5357 - val_loss: 2.5546 - val_accuracy: 0.2438
Epoch 6/30
200/200 - 17s - loss: 1.1141 - accuracy: 0.5585 - val_loss: 2.7448 - val_accuracy: 0.2275
Epoch 7/30
200/200 - 18s - loss: 1.0479 - accuracy: 0.5938 - val_loss: 2.7720 - val_accuracy: 0.2500
Epoch 8/30
200/200 - 18s - loss: 0.9915 - accuracy: 0.6068 - val_loss: 2.9227 - val_accuracy: 0.2488
Epoch 9/30
200/200 - 18s - loss: 0.9150 - accuracy: 0.6250 - val_loss: 2.8882 - val_accuracy: 0.2450
Epoch 10/30
200/200 - 18s - loss: 0.9104 - accuracy: 0.6332 - val_loss: 2.8632 - val_accura

In [14]:
# Make some predictions and evaluate
predictions = dl_model.predict(x=test_batches, verbose=0)
cl = test_batches.classes
pred_int = np.argmax(predictions, axis=-1)
acurr_score = accuracy_score(pred_int,cl)
prec_score = precision_score(pred_int,cl, average="macro")
print("Custom DL model ----> acurracy: ", acurr_score, " ,  precision: ", np.round(prec_score, 3))

Custom DL model ----> acurracy:  0.115  ,  precision:  0.102


In [ ]:
model = VGG16(weights='imagenet')
predictions = model.predict(x=test_batches, verbose=0)
cl = test_batches.classes
pred_int = np.argmax(predictions, axis=-1)
acurr_score = accuracy_score(pred_int,cl)
prec_score = precision_score(pred_int,cl, average="macro")
print("Custom DL model ----> acurracy: ", acurr_score, " ,  precision: ", np.round(prec_score, 3))

In [ ]:
file = f'./Lab05_Output/dataset/genReadyImages/test/car/203.jpg'
img = cv2.imread(file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# from tensorflow.keras.applications.mobilenet import MobileNet, decode_predictions, preprocess_input
image = cv2.resize(img, (224, 224))
prep = preprocess_input(np.array([image]))
# print(prep)
# cv2.imshow('window', prep[0])
# cv2.waitKey(0)
# pred = MobileNet().predict(x=prep)
pre = model.predict(x=prep)
from tensorflow.keras.applications.vgg16 import decode_predictions
print(decode_predictions(pre,top=3))